# Global-scale MODIS NDVI time series analysis

## 1. initial setting

In [1]:
from IPython.display import Image, display, HTML

%matplotlib inline

from pylab import *
import datetime
import math
import time

import ee
ee.Initialize()

## 2. functions for MODIS MOD13Q1 NDVI

 MODIS QA mask to filter out low quality pixels

In [2]:
def getQABits(image, start, end, newName):
    #Compute the bits we need to extract.
    p = 0
    for i in range(start,(end+1)):
        p += math.pow(2, i)

    # Return a single band image of the extracted QA bits, giving the band
    # a new name.
    return image.select([0], [newName])\
                  .bitwiseAnd(p)\
                  .rightShift(start)

#A function to mask out cloudy pixels.
def maskClouds(img):
  # Select the QA band.
    QA = img.select('DetailedQA')
  # Get the MOD_LAND_QA bits
    internalCloud = getQABits(QA, 0, 1, 'MOD_LAND_QA')
  # Return an image masking out cloudy areas.
    return img.mask(internalCloud.eq(0))

##originally function for landsat
#https://groups.google.com/forum/#!searchin/google-earth-engine-developers/python$20bitwiseAnd%7Csort:relevance/google-earth-engine-developers/OYuUMjFr0Gg/GGtYWh4CAwAJ
def maskBadData(image):
    invalid = image.select('DetailedQA').bitwiseAnd(0x6).neq(0)
    clean = image.mask(invalid.Not())
    return(clean)

def maskSummaryQA(img):
    QA = img.select('SummaryQA').eq(0)
    best = img.mask(QA)
    return(best)

## 3. Input Data (MODIS MOD13Q1 NDVI in 2009)

### 3.1 Input data 

in this demonstration the cloud mask is not applied.

In [3]:
MODcoll = ee.ImageCollection('MODIS/006/MOD13Q1') \
.filterDate('2001-01-01', '2001-12-31') \
.sort('system:time_start') \
.select('NDVI')

#.map(maskClouds) \
#.map(maskBadData)\
#.map(maskSummaryQA) \

count = MODcoll.size().getInfo()
print('count:', count);




('count:', 23)


### 3.2 see detailed information

toList(1,X). X=0,1,... first image as X=0, second as X=1....

check the info of first image

In [4]:
img1 = ee.Image(MODcoll.toList(1,0).get(0))
scale = img1.projection().nominalScale().getInfo()
props = img1.getInfo()['properties']
date = props['system:time_start']
system_time = datetime.datetime.fromtimestamp((date / 1000) - 3600) 
date_str = system_time.strftime("%Y_%m_%d")

img1 = img1.set('bands', date_str)

print('scale:', scale);
print('DATE:', date_str)


('scale:', 231.65635826395828)
('DATE:', '2001_01_01')


### 3.3 Calculate annual average of NDVI

In [5]:
img = MODcoll.mean()
props = img.getInfo()
print('property:', props)


('property:', {u'bands': [{u'crs': u'EPSG:4326', u'crs_transform': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0], u'id': u'NDVI', u'data_type': {u'max': 32767.0, u'type': u'PixelType', u'precision': u'double', u'min': -32768.0}}], u'type': u'Image'})


### 3.4 visualisation

In [6]:
regionfilter = ee.Geometry.Polygon([-170, 80, 0, 80, 170, 80, 170, -80, 10, -80, -170, -80]).toGeoJSON()

ndvi_palette = 'FFFFFF, CE7E45, DF923D, F1B555, FCD163, 99B718, 74A901, 66A000, 529400, 3E8601, 207401, 056201, 004C00, 023B01, 012E01, 011D01, 011301'
vizParams = {'min': -2000, 
             'max': 10000, 
             'region':regionfilter,
             'palette': ndvi_palette}


%config InlineBackend.figure_format = 'retina'    
print(img.getThumbUrl(vizParams))
Image(url=img.getThumbUrl(vizParams))


https://earthengine.googleapis.com/api/thumb?thumbid=1c137e89e1a12f5b38b4598a78d471f8&token=d8c12adab15680a057ab9453f1588a74


## 4. Trend analysis of annual average of NDVI in 2001-2016

### 4.1 parameters settings

In [7]:
#create name list

yr = range(2001, 2017)
yr = map(str, yr)
num=len(range(2001, 2017))
yy = np.array(["Y"]*num)
years = np.core.defchararray.add(yy, yr)

st = np.array(["-01-01"]*num)
ed = np.array(["-12-31"]*num)

starts = np.core.defchararray.add(yr, st)
ends = np.core.defchararray.add(yr, ed)

#print year
print("year:", years)
#print starts
print("starts:", starts)
#print ends
print("ends", ends)

('year:', array(['Y2001', 'Y2002', 'Y2003', 'Y2004', 'Y2005', 'Y2006', 'Y2007',
       'Y2008', 'Y2009', 'Y2010', 'Y2011', 'Y2012', 'Y2013', 'Y2014',
       'Y2015', 'Y2016'], 
      dtype='|S5'))
('starts:', array(['2001-01-01', '2002-01-01', '2003-01-01', '2004-01-01',
       '2005-01-01', '2006-01-01', '2007-01-01', '2008-01-01',
       '2009-01-01', '2010-01-01', '2011-01-01', '2012-01-01',
       '2013-01-01', '2014-01-01', '2015-01-01', '2016-01-01'], 
      dtype='|S10'))
('ends', array(['2001-12-31', '2002-12-31', '2003-12-31', '2004-12-31',
       '2005-12-31', '2006-12-31', '2007-12-31', '2008-12-31',
       '2009-12-31', '2010-12-31', '2011-12-31', '2012-12-31',
       '2013-12-31', '2014-12-31', '2015-12-31', '2016-12-31'], 
      dtype='|S10'))


### 4.2 Calculate annual NDVI

In [8]:
y = 0
MODcoll = ee.ImageCollection('MODIS/006/MOD13Q1') \
        .filterDate(starts[y], ends[y]) \
        .sort('system:time_start') \
        .select('NDVI')
start = starts[y]
end = ends[y]
avg = MODcoll.mean().rename([years[y]])

for y in range(1, 16):
    MODcoll = ee.ImageCollection('MODIS/006/MOD13Q1') \
        .filterDate(starts[y], ends[y]) \
        .sort('system:time_start') \
        .select('NDVI')
    start = starts[y]
    end = ends[y]
    average = MODcoll.mean()
    avg = avg.addBands(average.rename([years[y]])) 

In [9]:
info_bands = avg.getInfo()['bands']
#print('Dimensions:', info_bands[0]['dimensions'])
print('Number of bands:', len(info_bands))
##see bands
for ids in range(0,len(info_bands),1):
    print(info_bands[ids]['id'])

('Number of bands:', 16)
Y2001
Y2002
Y2003
Y2004
Y2005
Y2006
Y2007
Y2008
Y2009
Y2010
Y2011
Y2012
Y2013
Y2014
Y2015
Y2016


### 4.3 Mann-Kendall trend test

In [10]:
kndall_ans = avg.reduce(ee.Reducer.kendallsCorrelation(1))

In [11]:
info_bands = kndall_ans.getInfo()['bands']
#print('Dimensions:', info_bands[0]['dimensions'])
print('Number of bands:', len(info_bands))

##see bands
for ids in range(0,len(info_bands),1):
    print(info_bands[ids]['id'])

('Number of bands:', 2)
tau
p-value


### 4.4 Display the results

In [12]:
RdBu_palette = '#B2182B, #D6604D, #F4A582, #FDDBC7, #F7F7F7, #D1E5F0, #92C5DE, #4393C3, #2166AC'

kenn_tau = ee.Image(kndall_ans.select('tau')).multiply(10000).int16()
url = kenn_tau.getThumbUrl({
                'min':-10000,
                'max':10000,
                'region':regionfilter,
                'crs': 'EPSG:4326',
                'palette': RdBu_palette
    })    

%config InlineBackend.figure_format = 'retina'    
print(url)
Image(url=url)

https://earthengine.googleapis.com/api/thumb?thumbid=f670116d9d3d37412ce45f273cfb2c87&token=c5ae6cd19beb1caf0cfce5f8a537e326


### 4.5 Export Geotiff

In [ ]:
globalfilter = ee.Geometry.Polygon([-170, 80, 0, 80, 170, 80, 170, -80, 10, -80, -170, -80])
globalfilter = globalfilter['coordinates'][0]

task_config = {
    'description': 'imageToDriveExample',
    'scale': 231.65635826395828,  
    'region': globalfilter,
    'maxPixels': 5e10
    }

task = ee.batch.Export.image(kenn_tau, 'kenn_0116', task_config)
task.start()

In [ ]:
time.sleep(10)
while task.status()['state'] == 'RUNNING':
    print 'Running...'
    time.sleep(100)
print 'Done.', task.status()

Running...
